# Your info

Full name: Melika Nobakhtian

Student ID: 97522094

## Q4

In [ ]:
# Q4_graded

class FractionalNumber:
    def __init__(self, number: float):
        new_number = abs(round(number, 5))
        self.real_number = new_number
        self.decimal_number = int(new_number)
        self.float_number = int((new_number - int(new_number)) * (10**5))
        self.sign = -1 if number < 0 else 1

    @classmethod
    def from_binary_str(cls, binary_decimal: str, binary_float: str, sign : int):
        decimal = str(int(binary_decimal, 2))
        floating = str(int(binary_float, 2))
        return FractionalNumber(float(f'{decimal}.{floating}') * sign)

    def binary_form(self):
        return (str(bin(self.decimal_number).replace("0b", "")) , str(bin(self.float_number).replace("0b", "")))

    def __repr__(self):
        return str(self.real_number * self.sign)

In [ ]:
# Q4_graded

import random

class GeneticAlgorithmSearch:

    def __init__(self, equation, boundary, num_generations, population_size):
        self.keep_best = True
        self.num_generations = num_generations
        self.current_population = []
        self.best_so_far = None
        self.crossover_rate = 50  # 50%
        self.mutation_rate = 10  # 10%
        self.equation = equation
        self.lower_bound, self.upper_bound = boundary
        self.population_size = population_size

    
    def generate_initial_population(self):
        return [FractionalNumber(random.uniform(self.lower_bound, self.upper_bound)) for _ in range(self.population_size)]


    def handle_crossover_between(self, chromosome1, chromosome2):
        offspring_genes_decimal = ''.join(gene1 if random.randint(0, 100) < self.crossover_rate else gene2
                                  for gene1, gene2 in zip(chromosome1.binary_form()[0], chromosome2.binary_form()[0]))
        offspring_genes_float = ''.join(gene1 if random.randint(0, 100) < self.crossover_rate else gene2
                                  for gene1, gene2 in zip(chromosome1.binary_form()[1], chromosome2.binary_form()[1]))
        return FractionalNumber.from_binary_str(offspring_genes_decimal, offspring_genes_float, chromosome1.sign * chromosome2.sign)


    def handle_mutation_in(self, chromosome):
        flip_bit = lambda x: '0' if x == '1' else '1'
        binary_generator_decimal = (flip_bit(bit) if random.randint(0, 100) < self.mutation_rate else bit
                            for bit in chromosome.binary_form()[0])
        binary_generator_float = (flip_bit(bit) if random.randint(0, 100) < self.mutation_rate else bit
                            for bit in chromosome.binary_form()[1])
        return FractionalNumber.from_binary_str(''.join(binary_generator_decimal), ''.join(binary_generator_float), chromosome.sign)


    def should_exclude(self, chromosome):
        return self.lower_bound > chromosome.real_number > self.upper_bound

    def evaluate_chromosome(self, chromosome):
        return  - abs(self.equation(chromosome.real_number * chromosome.sign))

    def create_probabalistic_population_for_pick(self):
        """ best last """
        to_return = []
        for position, chromosome in enumerate(self.current_population):
            to_return.extend([chromosome]*position)
        return to_return

    def run_search(self):
        self.current_population = self.generate_initial_population()

        for i in range(self.num_generations):
            print('Starting generation {}'.format(i))
            
            # Evaluate
            self.current_population.sort(key=self.evaluate_chromosome)
            print("current population: ", self.current_population)
            self.best_so_far = self.current_population[-1]
            print('\tBest score so far = {}'.format(self.evaluate_chromosome(self.best_so_far)))
            print(f'best x so far : {self.best_so_far.real_number}')

            # Creating new population
            new_population = []

            # Copy best over if needed
            if self.keep_best:
                new_population.append(self.best_so_far)

            # Filling the rest
            probabilistic_population_for_mating = self.create_probabalistic_population_for_pick()
            while len(new_population) < len(self.current_population):
                parent1 = random.choice(probabilistic_population_for_mating)
                parent2 = random.choice(probabilistic_population_for_mating)

                # Performing crossover
                child = self.handle_crossover_between(parent1, parent2)

                # Performing mutation
                child = self.handle_mutation_in(child)

                # Ensuring child is good
                if self.should_exclude(child):
                    continue

                new_population.append(child)

            self.current_population = new_population

    def get_result(self):
        return self.best_so_far

In [ ]:
# Q4_graded
ga = GeneticAlgorithmSearch(equation=lambda x: (168* x * x * x -7.22 * x * x + 15.5 * x - 13.2), boundary=(-10, 10), num_generations=20, population_size=10)
ga.run_search()
best_estimate = ga.get_result()
print('Best Estimated X {}'.format(best_estimate.real_number))
print('Best Equation Value {}'.format(ga.equation(best_estimate.real_number)))

Starting generation 0
current population:  [-9.51073, -7.43652, 6.74634, -6.32827, -5.71989, -5.39079, -4.47453, 4.38043, -3.9964, 2.05945]
	Best score so far = -1455.548117455599
best x so far : 2.05945
Starting generation 1
current population:  [5.51346, -4.40326, 3.49262, 2.68923, -2.5998, 2.57884, 2.53676, 2.05945, 0.7195, 0.5907]
	Best score so far = -28.063228186224006
best x so far : 0.5907
Starting generation 2
current population:  [3.4926, 2.42524, 1.35088, 0.62878, 0.61258, 0.5907, 0.5899, 0.44695, 0.30624, 0.34493]
	Best score so far = -1.8181061594236212
best x so far : 0.34493
Starting generation 3
current population:  [1.22331, 0.9774, 0.61914, 0.61035, 0.59102, 0.29735, 0.30624, 0.33981, 0.34493, 0.36535]
	Best score so far = -0.30792466214699843
best x so far : 0.36535
Starting generation 4
current population:  [1.30462, 0.61148, 0.18036, 0.22334, 0.43118, 0.29702, 0.34617, 0.35997, 0.36535, 0.36791]
	Best score so far = -0.10836604012127005
best x so far : 0.36791
Star

## Q5


In [32]:
# Q5_graded
# Do not change the above line.
class Node:

    def __init__(self, name, machine, time_value, job, task_number):
        self.name = name
        self.job = job
        self.task_number = task_number
        self.machine = machine
        self.time_value = time_value
        self.successor_list = []
        self.predecessor_list = []
        self.pheromone_dict = {}
        self.start_time = None
        self.end_task = False

    def pheromones_get(self, _nodes):
        return {node: self.pheromone_dict[node] for node in _nodes if node in self.pheromone_dict}

    def nested_predecessors(self):
        nested_list = self.predecessor_list[:]
        if not nested_list:
            return []
        for node in nested_list:
            nested_list.extend(node.nested_predecessors())
        return nested_list

    def __repr__(self) -> str:
        return f'name: {self.name}, machine: {self.machine}, time: {self.time_value} , preds: {[node.name for node in self.predecessor_list]} , succ: {[node.name for node in self.successor_list]}'
        

In [33]:
# Q5_graded

def CreateNodelist(jobTimesMach):

  Nodelist = []

  # first we create start node
  start_node = Node(1, 0, 0, -1, -1)
  Nodelist.append(start_node)

  predecessors = []
  predecessors.append([])

  #create node for jobs
  node_name = 2
  for i, job in enumerate(jobTimesMach):
    prev_node = None
    for j, task in enumerate(job):
      machine_num = task[0]
      duration = task[1]
      new_node = Node(node_name, machine_num, duration, i+1, j+1)
      Nodelist.append(new_node)
      predecessors.append([])
      if prev_node is not None:
        predecessors[-1].append(prev_node)
      node_name += 1
      prev_node = new_node
    Nodelist[-1].end_task = True

  # Add successors and predecessors
  for idx, node in enumerate(Nodelist):
    if len(predecessors[idx]) > 0:
      for pred in predecessors[idx]:
        node.predecessor_list.append(pred)
        pred.successor_list.append(node)

  # Start Node Config
  start_nodes = [node for node in Nodelist if len(node.predecessor_list) == 0 and node.name != 1]
  start_node.successor_list = start_nodes
  for node in start_nodes:
    node.predecessor_list.append(start_node)

  # Last Node Config
  last_node = Node(node_name, 0, 0, -1, -1)
  last_nodes = [node for node in Nodelist if node.end_task]
  for node in last_nodes:
    last_node.predecessor_list.append(node)
    node.successor_list.append(last_node)

  Nodelist.append(last_node)

  return Nodelist

In [34]:
# Q5_graded

import random

class Ant:

    def __init__(self, _start_node):
        self.visited_list = [_start_node]
        self.visibility_list = []
        self.result_value = None
        self.visibility_list_update()

    def result_generate(self):
        while self.visibility_list:
            next_node = self.next_node_calculate()
            self.ant_move(next_node)
        self.result_value = result_value_calculate_as_makespan(self.visited_list)

    def next_node_calculate(self):
        pheromones_dict = self.visited_list[-1].pheromones_get(self.visibility_list)
        next_node = weighted_choice_sub(pheromones_dict)
        return next_node

    def ant_move(self, _next_node):
        self.visited_list.append(_next_node)
        self.visibility_list.remove(_next_node)
        self.visibility_list_update()

    def visibility_list_update(self):
        for successor in self.visited_list[-1].successor_list:
            if set(successor.predecessor_list).issubset(self.visited_list):
                self.visibility_list.append(successor)
    

def weighted_choice_sub(_dict):
    rnd = random.random() * sum(_dict.values())
    for i, w in enumerate(_dict):
        rnd -= _dict[w]
        if rnd < 0:
            return w


def result_value_calculate_as_makespan(_operations):
    for i, operation in enumerate(_operations):
        machine_unload_time = get_machine_unload_time(_operations[:i+1])
        if operation.predecessor_list:
            predecessor_end_times = [predecessor.start_time + predecessor.time_value for predecessor in
                                     operation.predecessor_list]
        else:
            predecessor_end_times = [0]
        operation.start_time = max([machine_unload_time] + predecessor_end_times)
    return _operations[-1].start_time + _operations[-1].time_value


def get_machine_unload_time(_operations):
    for operation in reversed(_operations[:-1]):
        if operation.machine == _operations[-1].machine:
            return operation.start_time + operation.time_value
    return 0

In [35]:
# Q5_graded

MULTIPLY = 0
ADD = 1


class AntAlgorithm:
    def __init__(self, _nodes_list):
        self.nodes_list = _nodes_list
        self.ant_population = []
        self.result_history = []
        self.init_pheromone_value = 0.8
        self.pheromone_potency = 0.7
        self.pheromone_distribution = 0.5
        self.max_min_ants_promoted = 5
        self.iterations = 30
        self.ant_population = []
        self.ant_population_size = 20
        self.evaporation_rate = 0.8

    def run(self):
        pass

    def pheromone_init(self):
        for node in self.nodes_list:
            nested_predecessors = [node] + node.nested_predecessors()
            for successor in self.nodes_list:
                if successor not in nested_predecessors:
                    node.pheromone_dict[successor] = self.init_pheromone_value

    @staticmethod
    def pheromone_trail_modify(_trail, _value, _operation):
        iterator = iter(_trail)
        node = next(iterator) 
        for next_node in iterator:
            if _operation == MULTIPLY:
                node.pheromone_dict[next_node] *= _value
            elif _operation == ADD:
                node.pheromone_dict[next_node] += _value
            node = next_node



class MaxMin(AntAlgorithm):
   
    def __init__(self, _nodes_list):
        AntAlgorithm.__init__(self, _nodes_list)
        self.history_best = Ant(self.nodes_list[0])
        self.history_best.result_value = 100000

    def run(self):
        self.pheromone_init()
        for iteration in range(self.iterations):
            self.ant_population = [Ant(self.nodes_list[0]) for _ in range(self.ant_population_size)]
            for ant in self.ant_population:
                ant.result_generate()
                self.evaporate_pheromone_trail(ant)

            self.graph_update()
            print(
                "running iteration: {0}, best result_permutation is: {1}".format(iteration,
                                                                                 self.result_history[-1].result_value))

    def graph_update(self):
        self.ant_population.sort(key=lambda x: x.result_value)
        self.result_history.append(self.ant_population[0])
        self.history_best = min(self.history_best, self.ant_population[0], key=lambda x: x.result_value)
        self.prepare_and_modify_best_trails()

    def prepare_and_modify_best_trails(self):
        self.pheromone_trail_modify(self.history_best.visited_list, 1 + self.pheromone_potency, MULTIPLY)
        for i in range(self.max_min_ants_promoted):
            value = (1 + self.pheromone_potency * (self.pheromone_distribution ** (i + 1)))
            self.pheromone_trail_modify(self.ant_population[i].visited_list, value, MULTIPLY)

    def evaporate_pheromone_trail(self, ant):
        self.pheromone_trail_modify(ant.visited_list, self.evaporation_rate, MULTIPLY)
        self.pheromone_trail_modify(ant.visited_list, 1 - self.evaporation_rate, ADD)


In [36]:
# Q5_graded

jobTimesMach = [[(1, 3), (2, 3), (3, 2)],
                [(1, 2), (3, 1), (2, 4)],
                [(2, 4), (3, 3)]]

Nodelist = CreateNodelist(jobTimesMach)
for node in Nodelist:
  print(node)
print("")
system =  MaxMin(Nodelist)
system.run()

print("result_permutation history:")
print([ant.result_value for ant in system.result_history])
best_result = system.history_best
print("best path: {0}".format(best_result.result_value))
print(" -> ".join([str(operation.name) for operation in best_result.visited_list]))

name: 1, machine: 0, time: 0 , preds: [] , succ: [2, 5, 8]
name: 2, machine: 1, time: 3 , preds: [1] , succ: [3]
name: 3, machine: 2, time: 3 , preds: [2] , succ: [4]
name: 4, machine: 3, time: 2 , preds: [3] , succ: [10]
name: 5, machine: 1, time: 2 , preds: [1] , succ: [6]
name: 6, machine: 3, time: 1 , preds: [5] , succ: [7]
name: 7, machine: 2, time: 4 , preds: [6] , succ: [10]
name: 8, machine: 2, time: 4 , preds: [1] , succ: [9]
name: 9, machine: 3, time: 3 , preds: [8] , succ: [10]
name: 10, machine: 0, time: 0 , preds: [4, 7, 9] , succ: []

running iteration: 0, best result_permutation is: 12
running iteration: 1, best result_permutation is: 11
running iteration: 2, best result_permutation is: 12
running iteration: 3, best result_permutation is: 11
running iteration: 4, best result_permutation is: 11
running iteration: 5, best result_permutation is: 11
running iteration: 6, best result_permutation is: 11
running iteration: 7, best result_permutation is: 11
running iteration: 8,

# <font color='red'>Submission</font>

1. Sign up in [Gradescope](https://www.gradescope.com) with proper name and student ID.
2. Fill in your full name (seperated by single spaces) and student ID in the beginning of this notebook.
3. After you're done with this notebook, you should do the following:
  - Clear all outputs of the notebook.
  ![clear all outputs](https://i.ibb.co/y6FrttB/Screen-Shot-2021-03-21-at-01-51-42.png)
  - Run all of the cells (if you skipped a question just leave the cell unchanged), and make sure all of your outputs are correct.
  ![run all](https://i.ibb.co/cgRcBZ0/Screen-Shot-2021-03-21-at-01-54-58.png)
  - Save your notebook.
  
  - If you're using Colab, download your notebook.
  ![download ipynb](https://i.ibb.co/2KxYM6K/Screen-Shot-2021-03-21-at-02-03-50.png)
  
  - Put the notebook file you just downloaded and `convert.py` in the same folder run the following command:
  ```bash
  python convert.py
  ```
  This will export your code for each question into a `.py` file.
    - **Note**: if you want to add more cells, add this to the **first** line of the cell:
  ```python
  # Q4_graded or #Q5_graded
  ```
  according to the question number.
  - There are 2 assignments in Gradescope:
  
    You should upload your **codes** and your **notebook** in `HW5` section and your final report for all of the questions as a **single pdf** file in `HW5 - Report`. Autograder will automatically check for:
    - `CI002_HW5.ipynb`
    - `Q4.py`
    - `Q5.py`
    - Your name and ID in the beginning of `.ipynb` file.

    It is important that you <font color='red'>**don't**</font> change the names of these files before submission.

4. If you pass the autograder, you're good to go.